# SPARQL Workshop - Section 03: Inserting and Updating Data

Before we get too far ahead, we should probably discuss how to insert and update data within Amazon Neptune.  There are also some best practices around loading data transactionally into Neptune in order to provide for high concurrency and throughput.  In this notebook we will cover the steps necessary to insert data, update (or upsert) data, and how to write queries for semi-bulk operations.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-rdf-updated.png"/>



## Create a resource with Properties

Each resource that you create in Neptune (weather in Property Graph/Gremlin or RDF/SPARQL) will have a unique ID (a resource identifier in the IMDb model), a label (or class), and a set of properties (or predicates). 

Whenever you're designing your data model, you should try wherever possible to use _predictable_ IDs – that is, IDs that you can later on predict when wanting to query for a specific vertex or edge. A good example of this is in the IMDb use case where we used the IMDb name and title IDs for each Artist and Production.

### 03.01 - Creating and adding a resource (vertex) using INSERT DATA

Create a new vertex representing yourself, with the ID 'person0'.  Instead of using Lorem Ipsom below, feel free to use your own name or make one up.

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX prop: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

INSERT DATA {
    resource:person0 rdf:type class:Person .
    resource:person0 prop:firstName 'Lorem' .
    resource:person0 prop:lastName 'Ipsom'
}

 Let's check our work.  By running the following query, we should return all predicates related to the person0 vertex.

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
        
SELECT * 
WHERE 
{
    resource:person0 ?p ?o
}

### 03.02 - More on Vertex / Resource IDs

Typically in RDF, you have an idea of how you want to define a unique identifier (URI) for each entity or vertex in your data model.  Conversely, the Property Graph framework tends to lend itself to some use cases where unique identifiers are something that need to be manufactured.  When adding data to an RDF model in Neptune, you will need to provide any unique identifiers (the full URIs to uniquely identify each entity).  For Property Graph, you can create vertices and edges where Neptune can auto-generate identifiers as UUIDs.

_NOTE: SPARQL does provide a built-in UUID function.  You can use this if you find yourself needing to create arbitrary vertex IDs.  https://www.w3.org/TR/sparql11-query/#func-uuid

All of that being said, you can use any manner of string values for an identifier.  In the query below, we create a node representing a new (fake) artist - "Sherlock Holmes".  We will use this node in further examples below.

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX prop: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

INSERT DATA {
    resource:artist0 rdf:type class:Artist .
    resource:artist0 prop:name 'Sherlock Holmes' .
    resource:artist0 prop:birthyear 1899
}

Let's check our work.  We should now see all of the predicates and values related to this new Sherlock Holmes resource.

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
        
SELECT ?p ?o

WHERE {
    resource:artist0 ?p ?o
}

### 03.03 - Overwriting property values and cardinality

Actually, we made a mistake. Sherlock Holmes, by most accounts, was born in 1854, not 1899. So we need to correct his birthday.  He also died in 1893, so let's make sure we add that as well.

Before we do that, you should understand how Neptune stores property values.  Be default, Neptune stores all property values as sets.  This means that if we run a new `INSERT` query with the new values, the new values will simply be added as another triple (Sherlock when then have two `birthyear` properties).  In order to update the existing data, we can use the `DELETE`-`INSERT`-`WHERE` query pattern.  This allows us to delete the incorrect triple, and subsequently add both the correct and new data.

Here we will update the birthyear property and add the deathyear property:

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX prop: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

DELETE 
{ 
   resource:artist0 prop:birthyear 1899 
}

INSERT 
{
    resource:artist0 rdf:type class:Artist .
    resource:artist0 prop:name 'Sherlock Holmes' .
    resource:artist0 prop:birthyear 1854 .
    resource:artist0 prop:deathyear 1893 .
} 

WHERE 
{
    resource:artist0 prop:birthyear 1899
}

 Let's check our work.  We should now see the updated birthyear predicate and new deathyear predicate.

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
        
SELECT * WHERE {
    resource:artist0 ?p ?o
}

### 03.04 Create a Relation Between Two Resources

Now that we have created a couple of resources, let's try to create an edge (or relationship) between them.  We do this by adding triples relating the two.  In the case of a Person vertex and Artist vertex, we have the 'follows' predicate that we can use to relate the two.  Here we will add a triple to create this relationship.

In the following traversal, we will add a Follows edge between Sherlock Holmes and the vertex representing you:

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

INSERT DATA 
{
    resource:person0 objProp:follows resource:artist0 .
}

Let's check our work by querying across the outgoing edge from your vertex.  In the following query, you start with the resource URI associated with `person0`, traverse across to the related `Artist` resource via the `follows` predicate, and then return all predicates related to the related Artist (Sherlock Holmes) resource:

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
        
SELECT * 
WHERE 
{
    resource:person0 objProp:follows ?artist .
    ?artist ?property ?values
}

You should see all of the properties associated with Sherlock Holmes.

### 03.05 - Conditional writes using INSERT-WHERE

Sometimes you'll want the ability to create a new resource or predicate only under certain conditions.  This is where the `INSERT`-`WHERE` statement comes into play. Or, we may need to do a query first to find a URI that we need as part of our `INSERT` statement.

An example of this maybe adding a property only if a vertex describing a certain entity already exists.  Let's say we want to add an address for Sherlock Holmes, but we want to make sure Sherlock Holmes exists. We also want to look up Sherlock's URI ahead of adding the new address.  Here, we can use `INSERT`-`WHERE` to complete this action:

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX prop: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

INSERT 
{
    ?artist prop:address '221B Baker Street London, England'
} 
WHERE 
{
    ?artist prop:name 'Sherlock Holmes'
}

Let's check our work.  You should now see the new address predicate.

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX prop: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
        
SELECT * 
WHERE 
{
    resource:artist0 ?property ?value
}

### 03.06 - Bulk write operations

Last, but not least, if loading data transactionally into Neptune, it is good practice to batch load data whenever possible.  As you've seen in previous examples, you can use `INSERT DATA` as a means to load multiple triples in a single transaction.  This allows you to load both resources, relations and properties all together. 

In the following example, we will create a small sample graph with 5 vertices and 5 edges.

Further documentation:

- Best practices for bulk transactional loads with Neptune -  https://github.com/aws-samples/aws-dbs-refarch-graph/tree/master/src/writing-from-amazon-kinesis-data-streams

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX hotel: <http://aws.amazon.com/neptune/vegasgraph/hotel/>
PREFIX bus: <http://aws.amazon.com/neptune/vegasgraph/bus/>

INSERT DATA 
{
    hotel:MGM rdf:type class:Hotel .
    hotel:MGM hotel:name "MGM" .
    hotel:Aria rdf:type class:Hotel .
    hotel:Aria hotel:name "Aria" .
    hotel:Venetian rdf:type class:Hotel .
    hotel:Venetian hotel:name "Venetian" .
    hotel:Wynn rdf:type class:Hotel .
    hotel:Wynn hotel:name "Wynn" .
    hotel:Encore rdf:type class:Encore .
    hotel:Encore hotel:name "Encore" .
    
    hotel:MGM bus:route hotel:Aria .
    hotel:Aria bus:route hotel:Venetian .
    hotel:MGM bus:route hotel:Wynn .
    hotel:Encore bus:route hotel:Aria .
    hotel:Venetian bus:route hotel:MGM
}

Let's check our work:

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX hotel: <http://aws.amazon.com/neptune/vegasgraph/hotel/>
        
SELECT * 
WHERE 
{
    ?hotel rdf:type class:Hotel .
    ?hotel ?property ?value
}

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX hotel: <http://aws.amazon.com/neptune/vegasgraph/hotel/>
PREFIX bus: <http://aws.amazon.com/neptune/vegasgraph/bus/>
        
SELECT ?fromname ?toname WHERE {
    ?from bus:route ?to .
    ?from hotel:name ?fromname .
    ?to hotel:name ?toname
}

### Review

In the previous notebook we covered the many different ways that you can Create and Update data within Neptune using SPARQL.  We also covered some of the best practices regarding how to use unique subject IDs and how to batch multiple writes into a single transaction.

### End of Section 03 - Move onto the next notebook.